In [7]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

In [8]:
from langchain_openai import ChatOpenAI
from langchain_groq import ChatGroq

def add(a:int,b:int)->int:
    """ 
    Add a and b

    Args:
        a: first integer
        b: second integer
    """
    return a+b

def multiply(a:int,b:int)->int:
    """ 
    Multiply a and b.

    Args:
        a: first int
        b: second int
    """
    return a*b 

def divide(a:int,b:int)->int:
    """ 
    Divide a and b.
    Args:
        a: first int
        b: second int
    """
    return a/b

tools = [add,multiply,divide]
llm = ChatGroq(model_name="qwen-2.5-32b")

llm_with_tools = llm.bind_tools(tools)

In [9]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from typing import Annotated
from langgraph.graph.message import add_messages

class MessageState(TypedDict):
    messages: Annotated[list[AnyMessage],add_messages]

In [10]:
from langchain_core.messages import HumanMessage,SystemMessage

# system message
sys_msg = SystemMessage(content="You are a helpful assisstant tasked with performing arithmetic on a set of inputs")

def assisstant(state:MessageState):
    return {"messages":[llm_with_tools.invoke([sys_msg]+state["messages"])]}

In [11]:
from langgraph.graph import START,END,StateGraph
from langgraph.prebuilt import tools_condition
from langgraph.prebuilt import ToolNode
from IPython.display import display,Image

builder = StateGraph(MessageState)

builder.add_node("assisstant",assisstant)
builder.add_node("tools",ToolNode(tools))

builder.add_edge(START,"assisstant")
builder.add_conditional_edges(
    "assisstant",
    tools_condition
)

builder.add_edge("tools","assisstant")

react_graph = builder.compile()

display(Image(react_graph.get_graph().draw_mermaid_png()))

SSLError: HTTPSConnectionPool(host='mermaid.ink', port=443): Max retries exceeded with url: /img/JSV7aW5pdDogeydmbG93Y2hhcnQnOiB7J2N1cnZlJzogJ2xpbmVhcid9fX0lJQpncmFwaCBURDsKCV9fc3RhcnRfXyhbPHA+X19zdGFydF9fPC9wPl0pOjo6Zmlyc3QKCWFzc2lzc3RhbnQoYXNzaXNzdGFudCkKCXRvb2xzKHRvb2xzKQoJX19lbmRfXyhbPHA+X19lbmRfXzwvcD5dKTo6Omxhc3QKCV9fc3RhcnRfXyAtLT4gYXNzaXNzdGFudDsKCXRvb2xzIC0tPiBhc3Npc3N0YW50OwoJYXNzaXNzdGFudCAtLi0+IHRvb2xzOwoJYXNzaXNzdGFudCAtLi0+IF9fZW5kX187CgljbGFzc0RlZiBkZWZhdWx0IGZpbGw6I2YyZjBmZixsaW5lLWhlaWdodDoxLjIKCWNsYXNzRGVmIGZpcnN0IGZpbGwtb3BhY2l0eTowCgljbGFzc0RlZiBsYXN0IGZpbGw6I2JmYjZmYwo=?type=png&bgColor=!white (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1020)')))

In [6]:
messages = [HumanMessage(content="Add 10 and 14.multiply the output by 2.divide the output by 5")]

react_graph.invoke({"messages":messages})

{'messages': [HumanMessage(content='Add 10 and 14.multiply the output by 2.divide the output by 5', additional_kwargs={}, response_metadata={}, id='e8636fe7-7f02-4e29-ac7a-3b7b561dd944'),
  AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_k6px', 'function': {'arguments': '{"a": 10, "b": 14}', 'name': 'add'}, 'type': 'function'}, {'id': 'call_nj5t', 'function': {'arguments': '{"a": 24, "b": 2}', 'name': 'multiply'}, 'type': 'function'}, {'id': 'call_sh8j', 'function': {'arguments': '{"a": 48, "b": 5}', 'name': 'divide'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 367, 'total_tokens': 446, 'completion_time': 0.19905601, 'prompt_time': 0.019439873, 'queue_time': 0.234397599, 'total_time': 0.218495883}, 'model_name': 'qwen-2.5-32b', 'system_fingerprint': 'fp_35f92f8282', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-cdc5f334-bbff-4217-a313-e03df1e7700c-0', tool_calls=[{'name': 'add', 'args': {'a': 10, 